In [15]:
import os, warnings
from dotenv import load_dotenv

os.environ['KMP_DUPLICATE_LIB_OK'] = "True"
warnings.filterwarnings('ignore')

load_dotenv('.env')

True

In [16]:
from langchain_ollama import OllamaEmbeddings
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [17]:
embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url='http://localhost:11434')
# ver "vector-storesipynb" como se llega a esto:
db_name = r'C:\Users\Jorge Moraga Calvo\Desktop\Python\langchain\operations_research'
vector_store = FAISS.load_local(db_name, embeddings=embeddings, allow_dangerous_deserialization=True)


In [18]:
question = "¿Cuáles son los efectos de la investigación operativa?"
# Retrieval
docs = vector_store.search(query=question, k=5, search_type='similarity')

In [19]:
docs # k=5 => 5 ids

[Document(id='7aa62d3b-0303-4a5e-a306-4051a13532b2', metadata={'producer': 'Adobe Acrobat 8.1', 'creator': 'Adobe Acrobat 8.1 Combine Files', 'creationdate': '2010-05-24T10:07:08-05:00', 'source': 'document_store/pdfs\\hillier_lieberman_proof.pdf', 'file_path': 'document_store/pdfs\\hillier_lieberman_proof.pdf', 'total_pages': 20, 'format': 'PDF 1.6', 'title': 'Introduccion a la investigacion de operaciones', 'author': 'Frederick S. Hillier', 'subject': '', 'keywords': '', 'moddate': '2010-07-07T13:08:58-05:00', 'trapped': '', 'page': 1}, page_content='como validación del modelo. En cierto sentido, la IO involucra la investigación científi ca creativa \nde las propiedades fundamentales de las operaciones. Sin embargo, es más que esto. La IO se ocupa \ntambién de la administración práctica de la organización. Por lo tanto, para tener éxito, también debe \nproporcionar conclusiones claras que el tomador de decisiones pueda usar cuando sea necesario.\nOtra característica de la investigaci

In [20]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k':5})
docs = retriever.invoke(question)
docs

[Document(id='7aa62d3b-0303-4a5e-a306-4051a13532b2', metadata={'producer': 'Adobe Acrobat 8.1', 'creator': 'Adobe Acrobat 8.1 Combine Files', 'creationdate': '2010-05-24T10:07:08-05:00', 'source': 'document_store/pdfs\\hillier_lieberman_proof.pdf', 'file_path': 'document_store/pdfs\\hillier_lieberman_proof.pdf', 'total_pages': 20, 'format': 'PDF 1.6', 'title': 'Introduccion a la investigacion de operaciones', 'author': 'Frederick S. Hillier', 'subject': '', 'keywords': '', 'moddate': '2010-07-07T13:08:58-05:00', 'trapped': '', 'page': 1}, page_content='como validación del modelo. En cierto sentido, la IO involucra la investigación científi ca creativa \nde las propiedades fundamentales de las operaciones. Sin embargo, es más que esto. La IO se ocupa \ntambién de la administración práctica de la organización. Por lo tanto, para tener éxito, también debe \nproporcionar conclusiones claras que el tomador de decisiones pueda usar cuando sea necesario.\nOtra característica de la investigaci

### RAG Prompt Template

In [21]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub

In [22]:
prompt = hub.pull("rlm/rag-prompt")
prompt # es lo mismo que la casilla de abajo, solo que viene de manera automática

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [23]:
prompt = """
Eres un asistente para tareas de respuesta a preguntas. 
Usa los siguientes fragmentos de contexto recuperados para responder la pregunta. 
Si no sabes la respuesta, simplemente di que no la sabes o que está fuera de tu alcance. 
Usa tres oraciones como máximo y mantén la respuesta concisa.
Pregunta: {question}
Contexto: {context}
Respuesta:"""

prompt = ChatPromptTemplate.from_template(prompt)
#prompt

In [24]:
llm = ChatOllama(model='llama3.2:latest', base_url='http://localhost:11434')


In [25]:
def format_docs(docs):
    return '\n\n'.join([doc.page_content for doc in docs])

context = format_docs(docs)
#print(context)

In [26]:
rag_chain = (
    {'context': retriever|format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [31]:
#question = "¿Qué es la investigación operativa?"
response = rag_chain.invoke(question)

In [32]:
print(response)

La investigación operativa es un campo que combina la ciencia y la ingeniería para resolver problemas complejos en áreas como la administración de recursos, la producción y el manejo de cadenas de suministro. Su objetivo es encontrar soluciones óptimas para problemas de planificación, optimización y control en organizaciones. En este sentido, la investigación operativa busca proporcionar conclusiones claras y prácticas que puedan ser utilizadas por tomadores de decisiones.
